In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))

from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# Demonstration of using IMM-Filter for Maneuver Detection

In [ ]:
import numpy as np
from filterpy.kalman import KalmanFilter, IMMEstimator
from filterpy.common import Q_discrete_white_noise, Saver

from bokeh.plotting import figure, show
from bokeh.resources import INLINE
import bokeh.io
bokeh.io.output_notebook(INLINE)

In [ ]:
T=0.06 # cycle time
N=600
randn=np.random.randn(2,N)

In [ ]:
# kf describing the CA model
def setup_kalman_CA(z_sigma_x, z_sigma_v, q_sigma_a, q_add_x):
    kf = KalmanFilter(dim_x=3, dim_z=2)
    kf.x = np.array([[10.],
                      [0.],
                      [0.]])         # initial state (location and velocity)

    kf.F = np.array([[1.,T,0.5*T**2],
                      [0.,1.,T],
                      [0.,0.,1]])    # state transition matrix

    kf.H = np.array([[1.,0.,0.],
                      [0.,1.,0.]])   # Measurement function
    kf.P *= 100.                     # state covariance matrix
    kf.R = np.array([[z_sigma_x**2,   0.],
                     [0.,   z_sigma_v**2]]) # measurement uncertainty
    var_a=q_sigma_a**2
    kf.Q = Q_discrete_white_noise(3, T, var_a) # process uncertainty
    kf.Q[0,0] += q_add_x

    return kf

def run_kalman_CA(kf):
    saver = Saver(kf)
    steps=np.arange(0,N)
    t=T*steps
    for i in steps:
        kf.predict()
        if i < 2*N/3:
            z=np.array([[10. + randn[0,i] * np.sqrt(kf.R[0,0])],
                        [0.  + randn[1,i] * np.sqrt(kf.R[1,1])]])
        else:
            z=np.array([[20. + randn[0,i] * np.sqrt(kf.R[0,0])],
                        [0.  + randn[1,i] * np.sqrt(kf.R[1,1])]])
        kf.update(z)
        saver.save() # save the filter's state

    saver.to_array()
    return t, saver

def run_imm(kf_smooth,kf_fast):
    filters = [kf_smooth, kf_fast]
    mu = [0.5, 0.5]  # each filter is equally likely at the start
    trans = np.array([[0.999, 0.001], [0.001, 0.999]])
    imm = IMMEstimator(filters, mu, trans)
    
    saver = Saver(imm)
    steps=np.arange(0,N)
    t=T*steps
    for i in steps:
        imm.predict()
        if i < 2*N/3:
            z=np.array([[10. + randn[0,i] * np.sqrt(kf_fast.R[0,0])],
                        [0.  + randn[1,i] * np.sqrt(kf_fast.R[1,1])]])
        else:
            z=np.array([[20. + randn[0,i] * np.sqrt(kf_fast.R[0,0])],
                        [0.  + randn[1,i] * np.sqrt(kf_fast.R[1,1])]])
        imm.update(z)
        saver.save() # save the filter's state

    saver.to_array()
    return t, saver

def setup_competing_filters():
    kf_smooth = setup_kalman_CA(z_sigma_x=0.5, z_sigma_v=3/3.6, q_sigma_a=1.2, q_add_x=0.0)
    kf_fast   = setup_kalman_CA(z_sigma_x=0.5, z_sigma_v=3/3.6, q_sigma_a=1.2, q_add_x=0.1)
    
    return kf_smooth, kf_fast


* simple stationary setup in x direction with state $\vec{x} = \left(x,v_x,a_x\right)^T$
 * ego_vehicle with constant velocity
 * object with v_rel=0 thus constant distance x


* ground truth data
 * constant but noisy distance measurement to object for 24s
 * sudden jump in distance by factor 2 (step response) for further 10s


* smooth/slow linear Kalman Filter with constant acceleration model
* noisy/fast filter with equal measurement and process noise and additive term in $Q_{xx}$
* IMM mixing both filters with very low transition probability smooth to fast

In [ ]:
kf_smooth, kf_fast = setup_competing_filters()
t,kf1 = run_kalman_CA(kf_smooth)
t,kf2 = run_kalman_CA(kf_fast)

kf_smooth, kf_fast = setup_competing_filters()
t,imm = run_imm(kf_smooth, kf_fast)

p0=figure(title='CA model: comparison KF and IMM', x_axis_label='time [s]', y_axis_label='object distance x', plot_width=800)

p0.line(t, kf1.z[:, 0, 0], color='#999999', line_dash='solid', legend_label='raw measurement')

p0.line(t, kf1.x[:, 0, 0], color='blue', legend_label='smooth')
p0.line(t, kf1.x[:, 0, 0]+3*np.sqrt(kf1.P[:,0,0]), color='blue', line_dash='dotted', line_width=0.5, legend_label='smooth')
p0.line(t, kf1.x[:, 0, 0]-3*np.sqrt(kf1.P[:,0,0]), color='blue', line_dash='dotted', line_width=0.5, legend_label='smooth')

p0.line(t, kf2.x[:, 0, 0], color='green', legend_label='fast')
p0.line(t, kf2.x[:, 0, 0]+3*np.sqrt(kf2.P[:,0,0]), color='green', line_dash='dotted', line_width=0.5, legend_label='fast')
p0.line(t, kf2.x[:, 0, 0]-3*np.sqrt(kf2.P[:,0,0]), color='green', line_dash='dotted', line_width=0.5, legend_label='fast')

p0.line(t, imm.x[:, 0, 0], color='red', legend_label='imm')
p0.line(t, imm.x[:, 0, 0]+3*np.sqrt(imm.P[:,0,0]), color='red', line_dash='dotted', line_width=0.5, legend_label='imm')
p0.line(t, imm.x[:, 0, 0]-3*np.sqrt(imm.P[:,0,0]), color='red', line_dash='dotted', line_width=0.5, legend_label='imm')
p0.line(t, 14+2*imm.mu[:,0], color='blue', line_width=0.5)
p0.line(t, 14+2*imm.mu[:,1], color='green', line_width=0.5)

p0.legend.location = "top_left"
p0.legend.click_policy="hide"

show(p0)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=kf1.z[:, 0, 0], 
                         line=dict(color='#999999', width=1, dash='solid'), name='raw measurements'))

fig.add_trace(go.Scatter(x=t, y=kf1.x[:, 0, 0], 
                         line=dict(color='blue', width=1, dash='solid'), name='smooth', legendgroup='group1'))
fig.add_trace(go.Scatter(x=t, y=kf1.x[:, 0, 0]+3*np.sqrt(kf1.P[:,0,0]), 
                         line=dict(color='blue', width=0.5, dash='dot'), name='smooth', legendgroup='group1', showlegend=False))
fig.add_trace(go.Scatter(x=t, y=kf1.x[:, 0, 0]-3*np.sqrt(kf1.P[:,0,0]), 
                         line=dict(color='blue', width=0.5, dash='dot'), name='smooth', legendgroup='group1', showlegend=False))

fig.add_trace(go.Scatter(x=t, y=kf2.x[:, 0, 0], 
                         line=dict(color='green', width=1, dash='solid'), name='fast', legendgroup='group2'))
fig.add_trace(go.Scatter(x=t, y=kf2.x[:, 0, 0]+3*np.sqrt(kf2.P[:,0,0]), 
                         line=dict(color='green', width=0.5, dash='dot'), name='fast', legendgroup='group2', showlegend=False))
fig.add_trace(go.Scatter(x=t, y=kf2.x[:, 0, 0]-3*np.sqrt(kf2.P[:,0,0]), 
                         line=dict(color='green', width=0.5, dash='dot'), name='fast', legendgroup='group2', showlegend=False))

fig.add_trace(go.Scatter(x=t, y=imm.x[:, 0, 0], 
                         line=dict(color='red', width=1, dash='solid'), name='imm', legendgroup='group3'))
fig.add_trace(go.Scatter(x=t, y=imm.x[:, 0, 0]+3*np.sqrt(imm.P[:,0,0]), 
                         line=dict(color='red', width=0.5, dash='dot'), name='imm', legendgroup='group3', showlegend=False))
fig.add_trace(go.Scatter(x=t, y=imm.x[:, 0, 0]-3*np.sqrt(imm.P[:,0,0]), 
                         line=dict(color='red', width=0.5, dash='dot'), name='imm', legendgroup='group3', showlegend=False))


fig.update_layout(
    title='CA model: comparison KF and IMM',
    xaxis_title='time [s]',
    yaxis_title='$\text{object distance} x$',
    width=800,
    yaxis = dict(
      scaleanchor = "x",
      scaleratio = 1,
    )
)
fig.show()